## Exercise Sheet No. 3

---

> Machine Learning for Natural Sciences, Summer 2022, Jun.-Prof. Pascal Friederich, pascal.friederich@kit.edu
> 
> Deadline: 08.05.2023, 8 am
>
>Container version 1.0.0
> 
> Tutor: luca.torresi@kit.edu  
> **Please ask questions in the forum and only contact the Tutor when there are issues with the grading**
---

**Topic**: This exercise sheet will focus on linear algebra, knn classifier, precision, recall, ROC curves and feature reduction. You will continue to use ``numpy`` methods.

Please add here your group members' names and student IDs. 

Names: Robin Maurer, 

IDs: 2462304, 


## 3.1 Distance function computation

For many feature descriptors and also for a specific implementation of the nearest neighbor classifier, the Euclidean mutual distance between a set of points must be computed. This serves as a measure on how two points are distinct or how far apart they are in Euclidean space. We will make use of this very often in the future. Therefore this exercise focus on an efficient implementation using numpy. You should gain a better unstanding of `numpy` methods.

**3.1.1** Implement a function ``dist_loop(A,B)`` to computes the Euclidean distance between all instances between two sets of points $A \subset \mathbb{R}^{D}$ and $B \subset \mathbb{R}^{D}$. The input should be two matrices of shape $N \times D$ and $M \times D$. Do this by using explicit python loops to find the distance $d_{ij} = || a_{i} - b_{j} ||$ between two points $a_{i} \in A$ and $b_{i} \in B$. The output should be a $N \times M$ distance matrix. For the calculation of the Euclidean distance you might want to use ``numpy.square()``, ``numpy.sum()`` and ``numpy.sqrt()``.

In [ ]:
import numpy as np
from numpy import random
import sys
n = 500
m = 1000
d = 3
A_data = np.reshape(random.rand(n*d),(n,d))
B_data = np.reshape(random.rand(m*d),(m,d))
print("A shape:", A_data.shape, "B shape:", B_data.shape)

In [ ]:
def dist_loop(A,B):
    if A.shape[-1] != B.shape[-1]:
        raise ValueError("Error A and B must have same last dimension but got", A.shape[-1], B.shape[-1])
    n = A.shape[0]
    m = B.shape[0]
    distances = np.zeros((n, m))
    
    for i in range(n):
        for j in range(m):
            diff = A[i] - B[j]
            distances[i, j] = np.sqrt(np.sum(np.square(diff)))
    
    return distances
    #raise NotImplementedError()

**3.1.2** Since loops are rather slow in python, and more efficient code is required for numeric operations, write another function ``dist_vec(A,B)`` to compute the distance relying
on vectorization with ``numpy`` methods. Consult https://www.safaribooksonline.com/library/view/python-for-data/9781449323592/ch04.html and https://softwareengineering.stackexchange.com/questions/254475/how-do-i-move-away-from-the-for-loop-school-of-thought for information on how to do this. 

Tip: There is more than one way to do it. For the most straightforward answer, you need to add an additional dimension to `np.arrays` via, for example, `expand_dims`. If two `np.arrays` do not have matching shape they are then automatically broadcasted by repeating their respective element along the axis in question. Beside broadcasting, also the [indexing rules](https://numpy.org/doc/stable/reference/arrays.indexing.html) (required for 3.2, 3.3) of `numpy` help with vectorization to avoid python loops. You must not use ``scipy`` or its methods for this task!

In [ ]:
def dist_vec(A,B):
    if A.shape[-1] != B.shape[-1]:
        raise ValueError("Error A and B must have same last dimension but got", A.shape[-1], B.shape[-1])
    
    
    
    A_squared = np.sum(np.square(A), axis=1)
    B_squared = np.sum(np.square(B), axis=1)
    AB = np.dot(A, B.T)
    
    distances = np.sqrt(np.expand_dims(A_squared, axis=1) + np.expand_dims(B_squared, axis=0) - 2 * AB)
    
    return distances
    #raise NotImplementedError()

Check that the two function return the same result. And check the output shape of the results.

In [ ]:
result_loop = dist_loop(A_data,B_data)
result_vec = dist_vec(A_data,B_data)
result_vec_shape = np.shape(result_vec) # Check shape of result_loop
result_loop_shape = np.shape(result_loop) # Check shape of result_vec

assert np.allclose(result_loop, result_vec)
assert result_vec_shape == result_loop_shape

print("Shape of result_loop:", result_loop_shape)
print("Shape of result_vec:", result_vec_shape)
#raise NotImplementedError()

Now compare the run times of the two implementations using jupyter's ``%timeit`` command or pythons ``time``. How much faster is the vectorized version?

In [ ]:
# measure time for loop
%timeit result_loop = dist_loop(A_data,B_data)
# measure time for vectorized
%timeit result_vec = dist_vec(A_data,B_data)

In [ ]:
# sanity test because I dont know how to read timeit. Frage: warum haben die eine unterschiedliche Anzahl an loops?
import time

start_time = time.time()
dist_loop(A_data, B_data)
dist_loop(A_data, B_data)
dist_loop(A_data, B_data)
dist_loop(A_data, B_data)
dist_loop(A_data, B_data)
dist_loop(A_data, B_data)
dist_loop(A_data, B_data)
end_time = time.time()
print("Runtime of dist_loop():", (end_time - start_time)/7, "seconds")

start_time = time.time()
dist_vec(A_data, B_data)
dist_vec(A_data, B_data)
dist_vec(A_data, B_data)
dist_vec(A_data, B_data)
dist_vec(A_data, B_data)
dist_vec(A_data, B_data)
dist_vec(A_data, B_data)
end_time = time.time()
print("Runtime of dist_vec():", (end_time - start_time)/7, "seconds")

In [ ]:
speed_up_factor = 420 # A rough estimation is okay.
# YOUR CODE HERE
#raise NotImplementedError()

Note: It is absolutely critical that you understand vectorization, and with this the indexing and broadcasting methods of numpy. Not only do we need fast performance, but also all deep learning frameworks are based on array or tensor operations just like you used with ``numpy``.

In [ ]:
# Test for grading
assert callable(dist_loop)

In [ ]:
assert callable(dist_vec)

In [ ]:
# Test for grading
assert result_loop is not None
assert result_vec is not None

In [ ]:
assert result_vec_shape is not None
assert result_loop_shape is not None

In [ ]:
# Test for grading
assert speed_up_factor > 0

## 3.2 Precision-Recall Curves

In this exercise, we use sklearn's digits dataset for an image classification experiment. Each image has 64 grey scale pixels. <br> 
We will use a small subset of the dataset as training set for a k-nearest neighbor classifier to classify the rest of the images depending on the digit they represent. To apply knn we need a measure of similarity: we will use the previously implemented distance function to compute the euclidian distance between couples of image vectors. <br> 
Let's start loading the dataset, splitting it in a train and a test set and having a lookat some images.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# read and prepare the digits data
digits = load_digits()
data = digits["data"]
images = digits["images"]
target = digits["target"]

# split data in train and test set
X_labeled, X_unknown, y_labeled, y_unknown = train_test_split(data, target, test_size=0.98, random_state=42)
print(f'Train set shape: {X_labeled.shape}; test set shape: {X_unknown.shape}\n\n')

# Show digits
print(f'Example of images from classes: {target[:3]}')
fig = plt.figure(figsize = (10,3))
plt.gray()
plt.subplot(1,3,1); plt.axis('off')
plt.imshow(images[0])
plt.subplot(1,3,2); plt.axis('off')
plt.imshow(images[1])
plt.subplot(1,3,3); plt.axis('off')
plt.imshow(images[2])
fig.tight_layout(); plt.show()

Knn is a simple non-parametric model. For each datapoint in the test set we want to build a list, sorted by a similarity measure, of all the datapoints in the train set. Then the output of the classifier is determined by the "votes" of the k closest train datapoints. For example we can simply select as predicted class the most common class among the closest neighbors. Or, in a binary classification setting, define the probability of an object to belong to a given class as the normalized number of positive votes and then predict positively if the probability is over a pre-set threshold.

Let's implement a simple Knn classifier that can work both for a multiclass setting and as binary classifier when we specify the digit of interest. We will need the binary classifier to discuss some common performance metrics later on.

We will use the Euclidean distance between pixel values as similarity measure:

$d(X_i,X_{i'} ) = || X_i - X_{i'}||_2$

To efficiently compute these distances, you should use vectorization from exercise 3.1. Let $D$
be the full dissimilarity matrix, i.e. $D_{i i'} = d(X_i,X_{i'})$. An ``np.argsort()`` of row $D_i$ now gives the similarity ordering of all images $X_{i'}$ in the train set relative to image $X_i$.


Note: If you didn't manage to solve 3.1 you can now use `scipy.spatial.distance.cdist`. But you do not get points for this solution in 3.1.

In [ ]:
from collections import Counter
from scipy.spatial.distance import cdist

class Knn_Classifier:
    def __init__(self, X_unknown, X_labeled, y_labeled):
        D = dist_vec(X_unknown, X_labeled)                # compute distances from each X_unknown to each X_labeled
        nn = np.argsort(D)               # order neighbors from closest to furthest
        self.nn_targets = y_labeled[nn]  # store labels (our 'votes') of the ordered neighbors (given by y_labeled)

#raise NotImplementedError()

    def set_k(self, k):
        self.knn_targets = self.nn_targets[:,:k]      # restrict list of labels to k closest neighbors

#raise NotImplementedError()
        
    def __call__(self, label: int = None, threshold: float = .5):
        counters = [Counter(self.knn_targets[i]) for i in range(self.knn_targets.shape[0])]
        if label is None:
            # multiclass classifier, for each data point it outputs the most "voted" class by the neighbors
            preds = np.asarray([c.most_common(1)[0][0] for c in counters])
            return preds
        else:
            # binary classifier for digit 'label'
            # computes the probability a given data point belongs to a class or to any of the others
            # as the number of votes for that class normalized over the total number of votes
            probs = np.asarray([c.get(label, 0)/sum(c.values()) for c in counters])
            # predicts as belonging to the class 'label' if the computed probability exceeds the threshold
            preds = (probs>=threshold).astype(int)
            return preds, probs

Let's run a multiclass knn classification setting the number of neighbors k to 5. 

To have an overview of the goodness of the classification we will print a confusion matrix. A confusion matrix is defined as the matrix $C$ such that $C_{ij}$ is equal to the number of observations known to be in group $i$ and predicted to be in group $j$. It can be easily computed using the function "confusion_matrix" from sklearn.metrics and we'll plot it using seaborn heatmaps.

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

knn = Knn_Classifier(X_unknown, X_labeled, y_labeled)
knn.set_k(5)
preds = knn()
print(np.shape(preds),np.shape(y_unknown))
cf_matrix = confusion_matrix(y_unknown, preds)

fig, ax = plt.subplots(figsize=(8,6))
ax = sns.heatmap(cf_matrix, annot=True, fmt='', cmap='Blues')
plt.xlabel('Predicted labels')
plt.ylabel('Ground truth labels')
plt.show()
plt.close()

Each row of the confusion matrix represents the instances in an actual class while each column represents the instances in a predicted class. Once produced, we can use it directly to compute some performance measures. We will have a look at two evaluation metrics used to assess the performance of a machine learning model in a binary classification problem: precision and recall. 

In a binary setting, so considering in our example one class against all other classes, we can define:
<ol>
<li><p>true positives (TP): number of objects belonging to the class that were $\textbf{correctly}$ predicted as belonging to it; </p>
<li><p> true negatives (TN): number of objects $\textbf{not}$ belonging to the class that were $\textbf{correctly}$ predicted as $\textbf{not}$ belonging to it; </p>
<li><p> false positives (FP): number of objects $\textbf{not}$ belonging to the class that were $\textbf{incorrectly}$ predicted as belonging to it; </p>
<li><p> false negatives (FN): number of objects belonging to the class that were $\textbf{incorrectly}$ predicted as $\textbf{not}$ belonging to it; </p>
</ol>

Given these four quantities we can define:
<ol>
<li><p>Precision, or positive predictive value, is the proportion of true positive predictions among all the positive predictions made by the model. In formulas $\text{precision} = \frac{\text{TP}}{\text{TP} + \text{FP}}$</p>

<li><p>Recall, also termed sensitivity or probability of detection, is the proportion of true positive predictions among all the actual positive cases in the dataset. In formulas $\text{recall} = \frac{\text{TP}}{\text{TP} + \text{FN}}$.</p>
</ol>


A high precision indicates that the model is making few false positive predictions, i.e., positive predictions are correct. A high recall indicates that the model is making few false negative predictions, i.e., the model is correctly identifying all positive cases in the dataset. Ideally we would like both of these measures to be equal to 1, meaning that all objects predicted as belonging to the class are actually from that class (precision=1) and all objects belonging to the class are correctly predicted as belonging to it (recall=1). In general, there is a trade-off between precision and recall: increasing one metric often comes at the expense of the other.  

Let's compute precision and recall of our multiclass knn classifier using the confusion matrix we computed above.

In [ ]:
for label in range(3):
    print(f'Performance of multiclass classifier for digit {label} VS all others:')
    # to access predictions relative to a given class i we select the i-th column of the confusion matrix
    column_counts = cf_matrix[:, label]
    # the i-th row corresponds to the images actually belonging to class i 
    row_counts = cf_matrix[label]

    TP = column_counts[label]
    FP = sum(column_counts) - column_counts[label]
    FN = sum(row_counts) - row_counts[label]
    
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)

    print(f'\t-True positives: {TP}')
    print(f'\t-False positives: {FP}')
    print(f'\t-False negatives: {FN}')
    print(f'\t-Precision: {precision:.2f}')
    print(f'\t-Recall: {recall:.2f}')

We have seen how to compute precision and recall with respect to each digit using the confusion matrix produced in our multiclass setting. We are now going to switch to binary classification and metrics to introduce a useful tool, the receiver operating characteristic curve, or ROC curve. The ROC curve is a graphical representation of the performance of a binary classifier system as its discrimination threshold is varied. It plots the true positive rate (TPR), which is another name for the recall, against the false positive rate (FPR) at various classification thresholds.

We need now to define a new measure. The FPR, also termed fall-out or false alarm ratio, measures the proportion of negative cases that are incorrectly classified as positive, in formulas $\text{recall} = \frac{\text{FP}}{\text{FP} + \text{TN}}$.

Let's implement a function that given a knn model, the label of interest and a classification threshold, computes all the performance measures we discussed until now.

In [ ]:
#Kontrolle:
def compute_binary_metrics(knn, label: int, ground_truth: np.ndarray, threshold: float = .3):
    metrics = {}
    preds, probs = knn(label, threshold)
    target = (ground_truth==label).astype(int)
    metrics['TP'] = np.sum((preds==1)&(target==1))               # true_positives
    metrics['FP'] = np.sum((preds==1)&(target==0))               # false_positives
    metrics['TN'] = np.sum((preds==0)&(target==0))               # true_negative
    metrics['FN'] = np.sum((preds==0)&(target==1))               # false_negatives
# YOUR CODE HERE
#raise NotImplementedError()
    metrics['precision'] = metrics['TP']/(metrics['TP']+metrics['FP']) if metrics['TP']>0 else 0
    metrics['recall'] = metrics['TP']/(metrics['TP']+metrics['FN']) if metrics['TP']>0 else 0
    metrics['fall-out'] = metrics['FP']/(metrics['FP']+metrics['TN']) # Fall-Out
# YOUR CODE HERE
#raise NotImplementedError()
    return metrics

Let's test our funciton and compare the metrics with the previously computed ones. 

What is different now, with respect to the multiclass setting, is that we can set the classification threshold. Check how changing the threshold changes the values of our metrics.

In [ ]:
knn.set_k(5)
for label in range(3):
    print(f'Performance of binary classifier for digit {label}:')
    metrics = compute_binary_metrics(knn, label, y_unknown, threshold=.3)
    for k, v in metrics.items():
        print(f'\t-{k}: {v:.2f}')

Now, we can use these binary metrics to compute overall multi-class metrics using the following aggregation methods:<ol>
<li><p>Micro-averaging: In micro-averaging, we aggregate the binary metrics over all the classes. This means that we sum up the number of true positives, false positives, false negatives, and true negatives over all the classes and then compute the performance metrics.</p>
<li><p>Macro-averaging: In macro-averaging, we compute the binary metrics, e.g. precision, for each class separately and then take the average across all the classes. Macro-averaging gives equal weight to all classes and is suitable when we want to evaluate the performance of each class separately.</p>
<li><p>Weighted averaging: In weighted averaging, we compute the binary metrics for each class separately and then take the weighted average across all the classes. The weight of each class is proportional to the number of samples from that class in the dataset. Weighted averaging is most suitable averaging technique when the dataset is unbalanced among the various classes.</p>
</ol>


Below we implemented a function to compute multi-class performance measures aggregating the corresponding binary metrics with weighted averaging. 

In [ ]:
def compute_multiclass_metrics(knn, ground_truth, threshold=.3, labels=np.arange(10)):
    metrics_list = []
    for label in labels:
        metrics_list.append(compute_binary_metrics(knn, label, ground_truth, threshold))
        weights = np.asarray([(ground_truth==label).sum() for label in labels])
    multiclass_metrics = {}
    for m in ['precision', 'recall', 'fall-out']:
        multiclass_metrics[m] = sum([weights[i]*metrics_list[i][m] for i in range(len(labels))])/weights.sum()
    return multiclass_metrics, metrics_list

In [ ]:
knn.set_k(5)
metrics, _ = compute_multiclass_metrics(knn, y_unknown, threshold=.3)
print(f'Multiclass knn performance, weighted average: ')
for k, v in metrics.items():
    print(f'\t-{k}: {v:.2f}')

As stated above, the ROC curve is generated by calculating the TPR and FPR for different threshold values, ranging from 0 to 1. As the threshold increases, the model becomes more conservative, classifying fewer instances as positive, which leads to a decrease in the FPR and TPR. Conversely, as the threshold decreases, the model becomes more aggressive, classifying more instances as positive, which leads to an increase in the FPR and TPR.

A perfect classifier would have a ROC curve that passes through the top-left corner of the plot (TPR=1, FPR=0), indicating a TPR of 100% and an FPR of 0%. The area under the ROC curve (AUC) is a measure of the overall performance of the classifier, with a value of 1 indicating a perfect classifier and a value of 0.5 indicating a classifier that is no better than random guessing.

The following code plots the Recall VS Precision (RvP) curve and the ROC curve (with AUC), so that we can see how modifying the threshold gives us classifiers with different trade-offs for the metrics.

In [ ]:
def plot_measures(knn, y_unknown, plot_type='ROC', all=True):
    measures = {'ROC':('fall-out', 'recall'), 'RvP': ('recall', 'precision')}
    measures = measures[plot_type]
    m0 = {k:[] for k in ['mc']+list(range(10))}
    m1 = {k:[] for k in ['mc']+list(range(10))}
    for threshold in np.arange(0,1,.1):
        mc, bin = compute_multiclass_metrics(
                        knn, 
                        y_unknown, 
                        threshold)
        m0['mc'].append(mc[measures[0]])
        m1['mc'].append(mc[measures[1]])
        for i in range(10):
            m0[i].append(bin[i][measures[0]])
            m1[i].append(bin[i][measures[1]])
    if all:
        for i in range(10):
            plt.plot(m0[i][::-1], m1[i][::-1], label='digit_'+str(i))
    plt.plot(m0['mc'][::-1], m1['mc'][::-1], label='multiclass')
    auc = ''
    if plot_type=='ROC':
        plt.plot([0, 1], [0, 1], linestyle='dashed', color='red', label='random_guess')
        auc = np.abs(np.trapz(y=m1['mc'][::-1], x=m0['mc'][::-1]))
        auc = f', AUC: {auc:.3f}'
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.xlabel(measures[0])
    plt.ylabel(measures[1])
    plt.title(plot_type+auc)
    plt.legend()
    plt.grid()
    plt.show()
    plt.close()

Let's plot some of the curves varying the number of neighbors we use for the classification.

In [ ]:
for k in [2, 5, 20]:
    print(f'K: {k}')
    knn.set_k(k)
    plot_measures(knn, y_unknown, all=False, plot_type='RvP')
    plot_measures(knn, y_unknown, all=False, plot_type='ROC')

In [ ]:
# Test for grading
knn = Knn_Classifier(X_unknown, X_labeled, y_labeled)
res = np.asarray([3, 8, 3, 3, 8, 9, 5, 5, 2, 9, 6, 7, 2, 2, 7, 8, 0, 1, 2, 0, 5, 6, 8, 1, 1, 6, 0, 7, 7, 4, 4, 4, 4, 4, 4])
assert np.all(knn.nn_targets[2]==res), 'visible test: something wrong in nn_targets'

knn.set_k(5)
assert knn.knn_targets.shape[-1]==5, 'visible test: something wrong in knn_targets shape'

#similar test as above, hidden

In [ ]:
# Test for grading
knn = Knn_Classifier(X_unknown, X_labeled, y_labeled)
knn.set_k(5)
metrics = compute_binary_metrics(knn, label=5, ground_truth=y_unknown, threshold=.3)
res = {'TP': 141, 'FP': 38, 'TN': 1545, 'FN': 38, 'precision': 0.7877094972067039, 'recall': 0.7877094972067039, 'fall-out': 0.024005053695514846}
for k in metrics.keys():
    assert metrics[k]==res[k], f'visible test: metric {k} is wrong'

#similar test as above, hidden

### 3.3 Dimensionality reduction with PCA

Now we will try to reduce the number of features or pixels to two and re-test the precision/recall curves from exercise 3.2.

We will try with an unsupervised method, namely via Principle Component Analysis (PCA). The task is to use `scikit-learn`'s implementation of the PCA to reduce the number of pixels to two and then to make a scatter plot of the new features, where the color of each point represents the digits number.  Which property should this scatterplot have in order for the new features to be especially suitable for similarity search?

Implement a pca model using the imported sklearn function and transform the image features retaining only 2 components.

In [ ]:
#Kontrolle
from sklearn.decomposition import PCA

# YOUR CODE HERE
pca = PCA(n_components=2)
new_features_pca = pca.fit_transform(X_unknown,y_unknown)
#raise NotImplementedError()
plt.figure(figsize=(9, 7))
for i in range(10):
    plt.scatter(new_features_pca [y_unknown==i, 0], new_features_pca [y_unknown==i, 1], label=str(i))
plt.xlabel('Dim 1')
plt.ylabel('Dim 2')
plt.title('2D Feature Space')
plt.legend()
plt.show()
plt.close()

In [ ]:
# Test for grading
#checking if correct number of components is retained in new_features_pca

We fixed the number of components to be retained by the PCA to 2, so that we could easily visualize them with the scatter plot. What would be more sensible is to select a number of components sufficient enough to represent the variance of the data, in such a way to keep almost all valuable information carried by our features, but hopefully smaller than the total number of features, so to avoid redundancy. 

That's what we are doing next.

In [ ]:
pca = PCA()
pca.fit(X_labeled)
explained_variance = np.cumsum(pca.explained_variance_ratio_)
n_dim = next(ind for ind, x in enumerate(explained_variance) if x > .9)
print(f'Original number of features: {X_labeled.shape[-1]}')
print(f'Using only {n_dim} dimensions is enough to explain the 90% of the variance in the data')

Let's plot now the ROC curves of the knn classifier trained using all features and of the knn classifier with PCA reduced features. The performance improves only slightly in this specific example, but PCA reduction is a useful method o avoid overfitting in settings where the dimension of the dataset is small with respect to the dimension of the feature spacet.

In [ ]:
print('ROC curve of knn classifier using all available features')
knn = Knn_Classifier(X_unknown, X_labeled, y_labeled)
knn.set_k(5)
plot_measures(knn, y_unknown, all=False, plot_type='ROC')

print(f'ROC curve of knn classifier using {n_dim} most "important" features')
pca = PCA(n_components=n_dim)
pca.fit(X_labeled)
knn = Knn_Classifier(pca.transform(X_unknown), pca.transform(X_labeled), y_labeled)
knn.set_k(5)
plot_measures(knn, y_unknown, all=False, plot_type='ROC')